## Rayleigh phase function
This notebook plots the Rayleigh phase function as a polar plot as evaluated by Mitsuba 2.

In [4]:
import numpy as np
import enoki as ek
import mitsuba

mitsuba.set_variant('scalar_mono_double')

from mitsuba.core import Float, Vector3f, Frame3f
from mitsuba.core.xml import load_string
from mitsuba.render import MediumInteraction3f, PhaseFunctionContext, TransportMode

import plotly.graph_objects as go

In [5]:
def sph_dir(theta, phi):
    st, ct = ek.sincos(theta)
    sp, cp = ek.sincos(phi)
    
    return Vector3f(cp * st, sp * st, ct)

In [6]:
# Load Rayleigh phase function
rayleigh = load_string("""<phase version='2.2.1' type='rayleigh'/>""")

# Create a dummy medium interaction to use for eval()
mi = ek.zero(MediumInteraction3f)
mi.wavelengths = 370.0 # Not actually used currently

# Create an independent random sampler
sampler = load_string("""<sampler version='2.2.1' type='independent'/>""")

# Set transport mode (Importance or Radiance)
ctx = PhaseFunctionContext(sampler, TransportMode.Radiance)

# Incident direction (note: flipped to match BSDF convention)
wi = Vector3f(0, 0, 1)
mi.wi = -wi
mi.sh_frame = Frame3f(wi)

# Specify an array of outgoing directions
n = 180
theta_o = np.linspace(0.0, ek.Pi, n)
phi_o = np.linspace(0.0, 0.0, n) # Phi is irrelevant for Rayleigh phase function
wo = np.array([sph_dir(theta, phi) for theta, phi in zip(theta_o, phi_o)])

# Evaluate phase function
pf = np.array([rayleigh.eval(ctx, mi, w) for w in wo])

In [20]:
# Mirror data to fill out entire polar plot
pf_mirror = np.append(pf, np.flip(pf))

data = go.Scatterpolar(
           r = pf_mirror,
           theta = np.linspace(0.0, 360.0, n * 2),
           mode = 'lines'
       )

layout = go.Layout(
    polar_radialaxis_range = [0.0, 0.15]
#     polar_angularaxis_rotation = 90
)

fig = go.Figure(data = [data], layout = layout)
fig